# RAG Agent: Intelligent Document Q&A System

A production-ready Retrieval-Augmented Generation (RAG) system powered by NVIDIA AI endpoints, LangChain, and FAISS vector store.

## Overview

This notebook demonstrates a complete RAG pipeline that:
- Loads and processes research papers from arXiv
- Creates semantic embeddings using NVIDIA's embedding models
- Stores documents in a FAISS vector database
- Retrieves relevant context for user queries
- Generates grounded, citation-backed responses using LLMs

## Features

- **Intelligent Retrieval**: Uses semantic search to find the most relevant document chunks
- **Context Reordering**: Applies long-context reordering to optimize retrieval quality
- **Grounded Generation**: Responses are strictly based on retrieved documents
- **Streaming Output**: Real-time response generation for better UX
- **Production Ready**: Modular design for easy deployment and integration

## 1. Environment Setup

Install required dependencies and configure the environment.

In [1]:
# Set NVIDIA API Key
import os
os.environ['NVIDIA_API_KEY'] = 'YOUR_NVIDIA_API_KEY'

# Install required packages
%pip install -q langchain langchain-nvidia-ai-endpoints gradio
%pip install -q arxiv pymupdf faiss-cpu

# Import core libraries
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

from operator import itemgetter
from functools import partial
import os

print("✓ Environment setup complete")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✓ Environment setup complete
✓ Environment setup complete


## 2. Initialize AI Models

Configure NVIDIA AI endpoints for embeddings and language models.

In [9]:
# Initialize embedding model for semantic search
embedder = NVIDIAEmbeddings(
    model="nvidia/nv-embed-v1",
    truncate="END"
)

# Initialize LLM for response generation
# ⚠️ IMPORTANT: Use only valid NVIDIA-hosted models:
# - "meta/llama-3.1-8b-instruct" (Fast, 8B parameters)
# - "meta/llama-3.1-70b-instruct" (Powerful, 70B parameters)
# - "nvidia/llama-3.1-nemotron-70b-instruct" (NVIDIA's fine-tuned version)
# - "mistralai/mixtral-8x7b-instruct-v0.1" (Mistral's MoE model)

instruct_llm = ChatNVIDIA(model="openai/gpt-oss-120b")
llm = instruct_llm | StrOutputParser()

print("✓ AI models initialized successfully")
print(f"  - Embedding Model: nvidia/nv-embed-v1")
print(f"  - LLM: meta/llama-3.1-70b-instruct")

✓ AI models initialized successfully
  - Embedding Model: nvidia/nv-embed-v1
  - LLM: meta/llama-3.1-70b-instruct


## 3. Load Documents & Create Vector Store

Load research papers from arXiv and create FAISS embeddings on the fly.

In [ ]:
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import zipfile
import shutil

# Define arXiv paper IDs to load (you can customize this list)
arxiv_ids = [
    "2005.11401",  # RAG: Retrieval-Augmented Generation
    "1706.03762",  # Attention Is All You Need (Transformers)
    "2304.08485",  # GPT-4 Technical Report
    "2307.09288",  # Llama 2 Paper
]

print("Loading papers from arXiv...")
all_docs = []

for arxiv_id in arxiv_ids:
    try:
        loader = ArxivLoader(query=arxiv_id, load_max_docs=1)
        docs = loader.load()
        all_docs.extend(docs)
        print(f"  ✓ Loaded: {docs[0].metadata.get('Title', arxiv_id)[:60]}...")
    except Exception as e:
        print(f"  ✗ Failed to load {arxiv_id}: {str(e)}")

print(f"\n✓ Total papers loaded: {len(all_docs)}")

# Split documents into chunks for better retrieval
print("\nSplitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = text_splitter.split_documents(all_docs)
print(f"✓ Created {len(chunks)} document chunks")

# Create FAISS vector store from documents
print("\nCreating embeddings and building vector store...")
docstore = FAISS.from_documents(chunks, embedder)
print(f"✓ Vector store created successfully")
print(f"  - Total vectors: {len(chunks)}")
print(f"  - Sample chunk: {chunks[0].page_content[:100]}...")

# Save the vector store locally
print("\nSaving vector store to disk...")
docstore.save_local("docstore_index")
print("✓ Vector store saved to 'docstore_index' folder")

# Create a zip file of the vector store
print("\nCreating zip archive...")
shutil.make_archive("docstore_index", 'zip', "docstore_index")
print("✓ Zip file created: 'docstore_index.zip'")
print("\n📦 You can download 'docstore_index.zip' and reuse it next time!")
print("   This saves time by avoiding re-downloading and re-embedding papers.")

Loading papers from arXiv...
  ✓ Loaded: Retrieval-Augmented Generation for Knowledge-Intensive NLP T...
  ✓ Loaded: Retrieval-Augmented Generation for Knowledge-Intensive NLP T...
  ✓ Loaded: Attention Is All You Need...
  ✓ Loaded: Attention Is All You Need...
  ✓ Loaded: Visual Instruction Tuning...
  ✓ Loaded: Visual Instruction Tuning...
  ✓ Loaded: Llama 2: Open Foundation and Fine-Tuned Chat Models...

✓ Total papers loaded: 4

Splitting documents into chunks...
✓ Created 577 document chunks

Creating embeddings and building vector store...
  ✓ Loaded: Llama 2: Open Foundation and Fine-Tuned Chat Models...

✓ Total papers loaded: 4

Splitting documents into chunks...
✓ Created 577 document chunks

Creating embeddings and building vector store...


## 3B. Load Existing Vector Store (Alternative)

**Use this cell if you already have a saved vector store zip file.**  
Skip this if you just created a new vector store in the cell above.

In [ ]:
import zipfile
import os

# Path to your saved vector store zip file
zip_file_path = "docstore_index.zip"

# Check if the zip file exists
if os.path.exists(zip_file_path):
    print(f"📦 Found existing vector store: {zip_file_path}")
    
    # Extract the zip file
    print("Extracting vector store...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")
    print("✓ Vector store extracted successfully")
    
    # Load the vector store
    print("\nLoading vector store from disk...")
    docstore = FAISS.load_local(
        "docstore_index",
        embedder,
        allow_dangerous_deserialization=True
    )
    
    print("✓ Vector store loaded successfully")
    print(f"  - Total vectors: {docstore.index.ntotal}")
    print("\n🚀 Ready to use! You can now proceed to build the RAG pipeline.")
    
else:
    print(f"❌ Error: Vector store file '{zip_file_path}' not found!")
    print("Please run the cell above to create a new vector store, or")
    print("upload your existing 'docstore_index.zip' file to this directory.")

## 4. Build RAG Pipeline

Create the complete RAG chain with retrieval and generation components.

In [4]:
# Utility: Format retrieved documents into readable context
def docs2str(docs, title="Document"):
    """Convert document list to formatted string with citations."""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str


# Define the chat prompt template
chat_prompt = ChatPromptTemplate.from_template(
    "You are a helpful document chatbot. Answer questions based solely on the provided context."
    " User question: {input}\n\n"
    "Retrieved Context:\n{context}\n\n"
    "Instructions:\n"
    "- Only use information from the retrieved context\n"
    "- Cite sources when making claims\n"
    "- Be conversational and clear\n"
    "- If context is insufficient, acknowledge limitations\n\n"
    "Question: {input}"
)


# Utility: Stream output from chain results
def output_puller(inputs):
    """Extract and yield the 'output' field from runnable results."""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

print("✓ Prompt templates configured")

✓ Prompt templates configured


In [5]:
# Step 1: Build the retrieval chain
# ====================================

# Initialize long-context reordering for better retrieval quality
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

# Create retriever from vector store (top-5 most relevant chunks)
doc_retriever = docstore.as_retriever(search_kwargs={'k': 5})


def _context_to_text(docs):
    """Convert retrieved documents to formatted text string."""
    if not docs:
        return (
            "No relevant passages found in the knowledge base. "
            "Please rephrase your question or ask about a different topic."
        )
    return docs2str(docs)


# Build context retrieval pipeline
context_getter = (
    itemgetter('input')
    | doc_retriever
    | long_reorder
    | RunnableLambda(_context_to_text)
)

# Complete retrieval chain: input -> {input, context}
retrieval_chain = (
    {'input': (lambda x: x)}
    | RunnableAssign({'context': context_getter})
)

print("✓ Retrieval chain built")

✓ Retrieval chain built


In [6]:
# Step 2: Build the generation chain
# ====================================

# Create response generation pipeline
response_chain = (
    {
        'input': itemgetter('input'),
        'context': itemgetter('context'),
    }
    | chat_prompt
    | llm
)

# Wrap output for streaming compatibility
generator_chain = {'output': response_chain} | RunnableLambda(output_puller)

print("✓ Generation chain built")

✓ Generation chain built


In [7]:
# Step 3: Combine into complete RAG pipeline
# =============================================

rag_chain = retrieval_chain | generator_chain

print("✓ Complete RAG pipeline assembled")
print("\nPipeline Architecture:")
print("  1. User Query → Embedding")
print("  2. Semantic Search → Top-K Documents")
print("  3. Context Reordering → Optimized Context")
print("  4. LLM Generation → Grounded Response")

✓ Complete RAG pipeline assembled

Pipeline Architecture:
  1. User Query → Embedding
  2. Semantic Search → Top-K Documents
  3. Context Reordering → Optimized Context
  4. LLM Generation → Grounded Response


## 5. Test the RAG Agent

Run sample queries to validate the pipeline.

In [10]:
# Example 1: Basic query with streaming output
print("=" * 70)
print("QUERY: What are the latest developments in large language models?")
print("=" * 70)

for token in rag_chain.stream("What are the latest developments in large language models?"):
    print(token, end="", flush=True)

print("\n" + "=" * 70)

QUERY: What are the latest developments in large language models?
According to the retrieved context, the latest developments in large language models (LLMs) have been remarkable, with several new models being proposed and released. Some notable developments include the rise of open-source models such as BLOOM (Scao et al., 2022), OPT (Zhang et al., 2022), and Falcon (Penedo et al., 2023), which have been shown to match the performance of closed-source competitors like GPT-3 (Brown et al., 2020) and Chinchilla (Hoffmann et al., 2022).

Additionally, there has been a focus on fine-tuning LLMs for specific tasks, such as dialogue use cases. For example, the Llama 2 model, developed by Meta, has been optimized for chat interfaces and has shown promising results in terms of helpfulness and safety (Touvron et al., 2023).

Another trend is the increasing use of large vision models, which are being used to solve tasks independently, with language playing a supporting role in mapping visual si

In [11]:
# Example 2: Custom query
query = input("Enter your question: ")

print(f"\n{'=' * 70}")
print(f"QUERY: {query}")
print("=" * 70)

for token in rag_chain.stream(query):
    print(token, end="", flush=True)

print("\n" + "=" * 70)


QUERY: what is the latest knowlege you have about disadvantagers of RAG
Based on the provided context, the potential downsides or disadvantages of RAG (Retrieval-Augmented Generation) models are:

1. **Misinformation and bias**: Wikipedia, or any external knowledge source, may contain errors or biases, which can be perpetuated by RAG models [14].
2. **Misuse for malicious purposes**: RAG models can be used to generate abusive, fake, or misleading content, impersonate others, or automate spam/phishing content, similar to concerns raised about GPT-2 [50].
3. **Job automation**: Advanced language models like RAG may lead to the automation of various jobs in the coming decades [16].
4. **Hallucinations**: Although RAG is more strongly grounded in factual knowledge than previous models, it can still "hallucinate" and generate responses that are not entirely factual.

These potential downsides are mentioned in the context as potential risks associated with RAG models, and the authors sugges

## 6. Interactive Chat Interface (Optional)

Launch a Gradio web interface for easier interaction with the RAG agent.

In [14]:
import gradio as gr


def chat_interface(message, history):
    """Process user message and return RAG response."""
    response = ""
    for token in rag_chain.stream(message):
        response += token
    return response


# Create Gradio interface
demo = gr.ChatInterface(
    fn=chat_interface,
    title="🤖 RAG Agent: Intelligent Document Q&A",
    description="Ask questions about research papers in the knowledge base. Responses are grounded in retrieved documents.",
    examples=[
        "What are the key findings in recent NLP research?",
        "Explain the concept of retrieval-augmented generation",
        "What improvements have been made to transformer models?",
    ],
    theme=gr.themes.Soft(),
)

# Launch the interface
demo.launch(share=False)

c:\Users\Anmol\Desktop\sem 5\projects\RAGagent\.venv\Lib\site-packages\gradio\chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
